In [1]:

import numpy as np
import mrcfile
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib as mpl
import zarr

In [2]:
filename = '/Users/atchen/Downloads/50451_downsampled_bin10.mrc'

mrc = mrcfile.mmap(filename, mode='r+')

In [3]:
mrc.data.shape

(659, 1533, 1017)

In [6]:
mrc.data[0, 0, 0]

numpy.int8

In [30]:
from numcodecs import Blosc

In [32]:
compressor = Blosc(cname='lz4', clevel=5, shuffle=Blosc.BITSHUFFLE)
z = zarr.array(np.array(mrc.data, dtype='uint8'), chunks=(64, 128, 128), compressor=compressor)
z

<zarr.core.Array (659, 1533, 1017) uint8>

In [33]:
z.info

Type,zarr.core.Array
Data type,uint8
Shape,"(659, 1533, 1017)"
Chunk shape,"(64, 128, 128)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=BITSHUFFLE, blocksize=0)"
Store type,zarr.storage.KVStore
No. bytes,1027421199 (979.8M)
No. bytes stored,1004139106 (957.6M)
Storage ratio,1.0


In [43]:
from scipy.ndimage import zoom

In [46]:
s0 = np.array(mrc.data, dtype='uint8')
s1 = zoom(s0, 0.5)

In [47]:
s1.shape

(330, 766, 508)

In [48]:
s1.dtype

dtype('uint8')

In [55]:
store = zarr.DirectoryStore('/Users/atchen/mrc/50451.zarr')
root = zarr.group(store=store, overwrite=True)
fibsem_group = root.create_group('fibsem-uint8')

"""
"axes": ["x", "y", "z"],
"units": ["nm", "nm", "nm"],
"pixelResolution": {
    "dimensions": [4, 4, 4],
    "unit": "nm"
},
"ordering": "F",
"scales": [
    [1, 1, 1],
    [2, 2, 2],
    [4, 4, 4],
    [8, 8, 8],
    [16, 16, 16],
    [32, 32, 32]
],
"multiscales": [
{
    "datasets": [
    {
        "path": "s0",
        "transform": {
        "axes": ["z", "y", "x"],
        "scale": [4.0, 4.0, 4.0],
        "translate": [0.0, 0.0, 0.0],
        "units": ["nm", "nm", "nm"]
        },
        "compression": {
        "type": "gzip",
        "useZlib": false,
        "level": -1
        },
        "blockSize": [128, 128, 64],
        "dataType": "uint8",
        "dimensions": [4000, 3500, 2000]
    },
...
"""

fibsem_group.attrs['axes'] = ['z', 'y', 'x']
fibsem_group.attrs['units'] = ['nm', 'nm', 'nm']
fibsem_group.attrs['pixelResolution'] = {'dimensions': [40, 40, 40], 'unit': 'nm'}
# fibsem_group.attrs['ordering'] = 'F'
# fibsem_group.attrs['scales'] = [[1, 1, 1], [2, 2, 2], [4, 4, 4], [8, 8, 8], [16, 16, 16], [32, 32, 32]]
fibsem_group.attrs['scales'] = [[1, 1, 1], [2, 2, 2]]
fibsem_group.attrs['multiscales'] = [{
    "version": "0.4",
    "name": "fibsem-uint8",
    "axes": [
        # {"name": "t", "type": "time", "unit": "millisecond"},
        # {"name": "c", "type": "channel"},
        {"name": "z", "type": "space", "unit": "nanometer"},
        {"name": "y", "type": "space", "unit": "nanometer"},
        {"name": "x", "type": "space", "unit": "nanometer"}
    ],
    'datasets': [
        {
        'path': 's0',
        "coordinateTransformations": [{
            "type": "scale",
            "scale": [40.0, 40.0, 40.0]
        }],
        'compression': {
            'type': 'blosc',
            'cname': 'lz4',
            'clevel': 5,
            'shuffle': 'bit',
        },
        'blockSize': [64, 128, 128],
        'dataType': 'uint8',
        'dimensions': list(s0.shape)
    },
    {
        'path': 's1',
        "coordinateTransformations": [{
            "type": "scale",
            "scale": [80.0, 80.0, 80.0]
        }],
        'compression': {
            'type': 'blosc',
            'cname': 'lz4',
            'clevel': 5,
            'shuffle': 'bit',
        },
        'blockSize': [64, 128, 128],
        'dataType': 'uint8',
        'dimensions': list(s1.shape)
    }
    ],
    "coordinateTransformations": [{
        "type": "scale",
        "scale": [1.0, 1.0, 1.0]
    }],
    "type": "gaussian",
}]




In [56]:
z0 = fibsem_group.array('s0', s0, chunks=(64, 128, 128), compressor=compressor)
z1 = fibsem_group.array('s1', s1, chunks=(64, 128, 128), compressor=compressor)

In [57]:
store.close()